### IMPORT LIBRARIES

In [3]:
from transformers import TFAutoModelForTokenClassification, ElectraTokenizerFast
from datasets import load_dataset, load_metric
import numpy as np
from transformers import DataCollatorWithPadding
import tensorflow as tf
import math

### MODEL-CONFIGURATION DETAILS

In [2]:
BASE_PATH = "pretrained_models/"
MODEL_PATH = BASE_PATH + "NegBioElectra/"
LR_RATE = 1e-5
CKPT_PATH = BASE_PATH + "NegBioElectra_bioscope_cue_model"
EPOCHS = 15

### LOAD MODEL & TOKENIZER

In [4]:
model = TFAutoModelForTokenClassification.from_pretrained(
    MODEL_PATH, num_labels=4,from_pt=True
)
tokenizer = ElectraTokenizerFast.from_pretrained(MODEL_PATH,max_length=256)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForTokenClassification: ['embeddings.position_ids']
- This IS expected if you are initializing TFElectraForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### LOAD THE DATASET

In [5]:
ds = load_dataset("csv", data_files={'train': 'data/bioscope_cue_train_data.csv','test':'data/bioscope_cue_test_data.csv'})
ds

Using custom data configuration default-6da9a3d6d725ad52
Reusing dataset csv (/home/studio-lab-user/.cache/huggingface/datasets/csv/default-6da9a3d6d725ad52/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 7504
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 450
    })
})

### PRE-PROCESS DATA

In [6]:
def convert_labels(examples):
    l = {'label':[list(np.array(i.split("|"),dtype="int32")) for i in examples['label']]} 
    l.update(tokenizer(examples['sentence'],truncation=True,max_length=256,padding=True))
    return l

In [7]:
encoded_dataset = ds.map(convert_labels, batched=True)

Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/csv/default-6da9a3d6d725ad52/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-9746689509cf22d5.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/csv/default-6da9a3d6d725ad52/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-b6ae481d2148ca8a.arrow


### TRAIN-TEST SPLIT

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer,max_length=256, return_tensors="tf",padding="max_length")

In [9]:
tf_train_dataset = encoded_dataset["train"].to_tf_dataset(
    columns=['token_type_ids', 'attention_mask', 'input_ids'],
    label_cols=["label"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)
tf_validation_dataset = encoded_dataset["test"].to_tf_dataset(
    columns=['token_type_ids', 'attention_mask', 'input_ids'],
    label_cols=["label"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator,
)

In [8]:
list(tf_train_dataset.take(1))

[({'attention_mask': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
   array([[1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0]])>,
   'input_ids': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
   array([[    2,  1682,  3219, ...,     0,     0,     0],
          [    2,  1680,  1800, ...,     0,     0,     0],
          [    2,  1805, 17410, ...,     0,     0,     0],
          ...,
          [    2,  3666,  1685, ...,     0,     0,     0],
          [    2,  5794,    16, ...,     0,     0,     0],
          [    2,  1701,  4184, ...,     0,     0,     0]])>,
   'token_type_ids': <tf.Tensor: shape=(32, 256), dtype=int64, numpy=
   array([[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
     

### MODEL TRAINING

In [9]:
optimizer = tf.keras.optimizers.Adam(learning_rate=LR_RATE, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [10]:
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=CKPT_PATH,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    save_weights_only=True)
callbacks = [model_checkpoint_callback]

In [11]:
history = model.fit(
 tf_train_dataset, 
 epochs=EPOCHS, 
 validation_data=tf_validation_dataset, 
 callbacks =[callbacks]
)

Epoch 1/15
234/234 [==============================] - 394s 2s/step - loss: 0.3844 - accuracy: 0.9086 - val_loss: 0.0770 - val_accuracy: 0.9870
Epoch 2/15
234/234 [==============================] - 371s 2s/step - loss: 0.0689 - accuracy: 0.9893 - val_loss: 0.0742 - val_accuracy: 0.9870
Epoch 3/15
234/234 [==============================] - 374s 2s/step - loss: 0.0637 - accuracy: 0.9893 - val_loss: 0.0617 - val_accuracy: 0.9870
Epoch 4/15
234/234 [==============================] - 373s 2s/step - loss: 0.0550 - accuracy: 0.9899 - val_loss: 0.0539 - val_accuracy: 0.9911
Epoch 5/15
234/234 [==============================] - 372s 2s/step - loss: 0.0369 - accuracy: 0.9924 - val_loss: 0.0076 - val_accuracy: 0.9992
Epoch 6/15
234/234 [==============================] - 371s 2s/step - loss: 0.0083 - accuracy: 0.9990 - val_loss: 0.0050 - val_accuracy: 0.9992
Epoch 7/15
234/234 [==============================] - 367s 2s/step - loss: 0.0058 - accuracy: 0.9993 - val_loss: 0.0043 - val_accuracy: 0.9993

### MODEL EVALUATION BASED ON F1-SCORE

In [10]:
model.load_weights(CKPT_PATH)

In [11]:
tf_validation_dataset_ = encoded_dataset["test"].to_tf_dataset(
    columns=['token_type_ids', 'attention_mask', 'input_ids'],
    label_cols=["label"],
    shuffle=True,
    batch_size=1,
    collate_fn=data_collator,
)

In [12]:
def sigmoid_array(x):                                        
    return 1 / (1 + np.exp(-x))

In [13]:
inp = []
pred_out = []
true_out = []
for i in tf_validation_dataset_:
    in_,out_ = list(i)
    inp += list(in_['input_ids'].numpy())
    true_out += list(out_.numpy())
    pred = model(in_)
    pred_out += list(np.argmax(pred['logits'].numpy(),axis=-1))
    
len(inp),len(true_out),len(pred_out)

(450, 450, 450)

In [14]:
pred_out = np.array(pred_out)
pred_out.shape

(450, 256)

In [15]:
true_out = np.array(true_out)
true_out.shape

(450, 256)

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

m = MultiLabelBinarizer().fit(true_out)

f1_score(m.transform(true_out),
         m.transform(pred_out),
         average='weighted',zero_division=1)

0.9901961095052481

### ILLUSTRATION OF MODEL PREDICTING CUE.

In [13]:
sents = ["I dont like cake","she was not diagnosed with cancer","he failed to have plural effusion"]

In [14]:
from transformers import TokenClassificationPipeline

In [15]:
pipe = TokenClassificationPipeline(model=model, tokenizer=tokenizer)

In [16]:
out = pipe(["May month is good","she may have a lump","she was not diagnosed with cancer","he failed to have plural effusion","her diagnosis neither suggests tumour nor indicates carcinoma"])



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
from termcolor import colored
print(colored("Sample model output","blue"))
print(f"{'*'*10} NOTE: {colored('GREEN','green')} indicates Speculation, {colored('RED','red')} indicates Negation {'*'*10}")
print()
for j in out:
    s = []
    for i in j:
        l = i['word']
        if i['entity'] == 'LABEL_1':
            s.append(l)
        elif i['entity'] == 'LABEL_2':
            s.append(colored(l,'red'))
        elif i['entity'] == 'LABEL_3':
            s.append(colored(l,'green'))
    text = ' '.join([x for x in s])
    print(text.replace(' ##',''))
    

Sample model output
********** NOTE: GREEN indicates Speculation, RED indicates Negation **********

may month is good
she may have a lump
she was not diagnosed with cancer
he failed to have plural effusion
her diagnosis neither suggests tumour nor indicates carcinoma
